# 2019-02-04 - Protocol V2
Sans artéfacts et avec des écrans noirs au début et à la fin

In [1]:
import os
import numpy as np
import MotionClouds as mc
import datetime
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
import imageio
import warnings
from joblib import Parallel, delayed

downscale = 1
fx, fy, ft = mc.get_grids(mc.N_X//downscale, mc.N_Y//downscale, mc.N_frame//downscale)

now = datetime.datetime.now()
strtime = now.strftime("%Y-%m-%d_%H:%M:%S")   

mc.figpath = os.path.join('sequences/MC%s'% strtime)
if not(os.path.isdir(mc.figpath)): os.mkdir(mc.figpath)

The [Exponential distribution](https://en.wikipedia.org/wiki/Exponential_distribution) is the probability distribution that describes the time between events in a Poisson point process

In [ ]:
def make_one_block(N_X, N_Y, seed, B_thetas, N_frame_total,  
                   N_frame_mean=6, N_theta=12, contrast=1.,
                  refresh_rate = 120, duration = 60,mean_duration = 3):
    fx, fy, ft = mc.get_grids(N_X, N_Y, 1)

    rng = np.random.RandomState(seed)

    N_frame = 0
    im = np.zeros((N_X, N_Y, 0))
    disk = mc.frequency_radius(fx, fy, ft) < .5
    itr = 0
    
    if duration != None and mean_duration != None and refresh_rate != None :
        N_frame_total = refresh_rate * duration
        N_frame_mean = refresh_rate * mean_duration
    
    while N_frame < N_frame_total:
        itr +=1
        N_frame_sub = int(rng.exponential(N_frame_mean))+1
        theta = np.int(rng.rand()*N_theta) * np.pi / N_theta
        B_theta = B_thetas[rng.randint(len(B_thetas))]
        mc_i = mc.envelope_gabor(fx, fy, ft, 
                                         V_X=0., V_Y=0., B_V=0., 
                                         sf_0=sf_0, B_sf=B_sf, 
                                         theta=theta, B_theta=B_theta)
        im_ = np.zeros((N_X, N_Y, 1))
        im_ += mc.rectif(mc.random_cloud(mc_i, seed=seed+N_frame), contrast=contrast)
        im_ *= disk # masking outside the disk 
        im_ += .5*(1-disk) # gray outside the disk
        im_ = im_ * np.ones((1, 1, N_frame_sub)) #  expand to N_frame_sub frames

        im_[6:20,6:20,:] = 0
        if itr % 2 == 0 :
            im_[6:20,6:20,-3:-1] = 1
        else :
            im_[6:20,6:20,-1] = 1 
        im = np.concatenate((im, im_), axis=-1) # montage
        '''im /= im.max()
        im *= 255
        im /= 4
        im = im.astype(np.uint8)'''
        N_frame = im.shape[-1]

        #data appending
        with open(mc.figpath+'/'+'sequence.csv', "a+") as writer_file:
            data_writer = csv.writer(writer_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            data_writer.writerow([itr, N_frame, N_frame/refresh_rate, 
                                  theta, B_theta, sf_0, B_sf, N_X, N_Y, N_frame_total, refresh_rate])

    return im[:, :, :N_frame_total], itr

#Parameters
N_X = fx.shape[0]
width = 29.7*256/1050 
N_X, N_Y = mc.N_X, mc.N_Y

N_frame_total = 200
N_frame_mean = 12
N_theta = 12

sf_0 = 4.*width/N_X
B_V = .0     # BW temporal frequency (speed plane thickness)
B_sf = sf_0   # BW spatial frequency

B_thetas = [np.pi/32, np.pi/16, np.pi/8, np.pi/4] #bandwidth opening choices
seed = 42 #standardized random vector

#Overrides N_frame total, nframe mean
refresh_rate = 24 #Hz
duration = 1800 #s
mean_duration = 10 #s

'''refresh_rate = None #Hz
duration = None #s
mean_duration = None #s'''


#refresh rate and duration and mean duration override the N_frame total;N_frame_mean unless left to None
im = make_one_block(N_X, N_Y, seed=seed, B_thetas=B_thetas, 
                    N_frame_total=N_frame_total, N_frame_mean=12, N_theta=12,
                   refresh_rate = refresh_rate, duration = duration, mean_duration = mean_duration)

warnings.filterwarnings("ignore")
for i in range(250):
    imageio.imwrite(mc.figpath+'/%s.png'% i, np.zeros_like(im[0][:,:,0]))
    
for i in range(im[0].shape[-1]+250, im[0].shape[-1]+500):
    print(i)
    imageio.imwrite(mc.figpath+'/%s.png' %i, np.zeros_like(im[0][:,:,0]))
    
for i in range(im[0].shape[-1]):
    imageio.imwrite(mc.figpath+'/%s.png' % (i+250) , np.rot90(im[0][:,:,i]))

In [ ]:
pixvals = []
for i in range(im[0].shape[-1]+500):
    pixval = imageio.imread('./sequences/MC2019-02-05_11:51:42/%s.png'%i)[0,0]
    pixvals.append(pixval)

In [ ]:
plt.plot(pixvals)

In [ ]:
plt.imshow(im[0][:,:,0])

# Protocol description

In [ ]:
print('Total duration of %s frames, which is %.3fs at %s Hz' % (N_frame_total, N_frame_total/refresh_rate, refresh_rate))
print('A total of %s blocks where generated, with the mean duration of each block being %s frames' % (im[1],N_frame_mean))
print('%s differents angles where shown, with %s possible bandwiths' % (N_theta, len(B_thetas)))
print('-----------------\nAll files were saved in /%s with format %s' % (mc.figpath, vext))

# Adapting the spatial frequency parameters for the screen (and the mouse)

My version :

In [ ]:
object_px = N_X #px
dpi = 72 #dot per inches
pixel_per_cm = dpi* .393701 #pixel per cm
distance = 60 #cm

object_size_cm = object_px/pixel_per_cm #cm

object_size_angles = np.degrees(2* np.arctan( (object_size_cm/2)/distance))
deg_per_px = object_size_angles/N_X
print('The size of the MC is %s degrees' % object_size_angles)
print('The size of a single pixel is %s degrees' % (object_size_angles/N_X))

Laurent's (probably the one that's correct)

viewingDistance = 60 # cm A MODIFIER
screen_width_cm = 30 # cm A MODIFIER

print('Visual angle of the screen', 2*np.arctan(screen_width_cm/2/viewingDistance)*180/np.pi)
print('Degrees per centimeter', 2*np.arctan(screen_width_cm/2/viewingDistance)*180/np.pi/screen_width_cm)

screen_width_px = 1600 # pixels 
screen_height_px = 900 # pixels

deg_per_px = 2*np.arctan(screen_width_cm/2/viewingDistance)*180/np.pi/screen_width_px
print('Degrees per pixel', deg_per_px)

stim_size = deg_per_px*N_X
print('Stimulus angular size', stim_size)

The central spatial frequency ``sf_0`` is defined as the frequency (number of cycles) *per pixel*, so that to get 

In [ ]:
print('width of these motion clouds (', N_X, ', ', N_Y, ')')
print('width of stimulus in degrees', N_X * deg_per_px)
phi_sf_0 = .5 # Optimal spatial frequency [cpd]
print('Optimal spatial frequency in cycles per degree', phi_sf_0)
print('Optimal spatial frequency in cycles per window = ', phi_sf_0 *  N_X * deg_per_px)
sf_0 = phi_sf_0 * deg_per_px
print('cycles per pixel = ', sf_0)

Similarly the spatial frequeny bandwidth as a function of the experimental parameters:

In [ ]:
phi_sf_0 = .5 # Optimal spatial frequency [cpd] https://www.sciencedirect.com/science/article/pii/S0042698904004390
phi_B_sf = 2. # Optimal spatial frequency bandwidth [in octaves]
B_Sf = sf_0 # good qualitative approximation

In [ ]:
phi_B_V = 5. # Optimal temporal frequency bandwidth [Hz]

#tf_opt = 1 # Hz
T = 0.250            # Stimulus duration [s] 
framerate = 100.    # Refreshing rate in [Hz]
Bv = phi_B_V # good qualitative approximation 

In one script:

import numpy as np
import MotionClouds as mc
import os

# Clouds parameters in absolute units
N_X = 512
width = 29.7*N_X/1050
phi_sf_0 = .5 # Optimal spatial frequency [cpd]

sf_0 = phi_sf_0*width/N_X
B_sf = sf_0   # BW spatial frequency
B_V = .5     # BW temporal frequency (speed plane thickness) WARNING temporal autocorrelation depends on N_frame

# generate zip files
dry_run = True
dry_run = False
      
for seed in [2016 + i for i in range(7)]:
    name_ = name + '_seed_' + str(seed)
    if not dry_run:
        if  not(os.path.isfile(os.path.join(mc.figpath, name_ + vext))):
            im = make_one_block(N_X, N_X, seed=seed, B_thetas=B_thetas, N_frame_total=200, N_frame_mean=25, N_theta=12)
            mc.anim_save(mc.rectif(im, contrast=contrast), os.path.join(mc.figpath, name_), vext=vext)
        else:
            print(' MC ' + os.path.join(mc.figpath, name_) + ' already done')
    else:
        print(' MC ' + os.path.join(mc.figpath, name_) + ' skipped  (dry run)')
